In [1]:
# import pandas as pd
import MagmaPandas as mp
import MagmaPandas.geochemistry.PEC as pec
import pandas as pd
import elements as e
import matplotlib.pyplot as plt
import meltInc.plotting as p

p.layout()
config = pec.PEC_configuration
config.print()

############## Post-entrapment modification #############
#################### correction model ###################

Settings________________________________________________
Fe2+ behaviour...................................buffered
Stepsize equilibration (moles)...................0.002   
Stepsize crystallisation (moles).................0.05    
Decrease factor..................................5       
FeO convergence (wt. %)..........................0.05    
Kd convergence...................................0.001   
Temperature convergence (°)......................0.1     


In [2]:
melt = mp.read_melt('./melt.csv', index_col=['name'], total_col='total')
olivine = mp.read_olivine('./olivine.csv', index_col=['name'], total_col="total")

In [3]:
samples = ["PI041-04-06", "PI041-02-01", "PI041-02-02"]
inclusions = melt.copy()
ol_host = olivine.copy()
forsterites = ol_host.forsterite
Fo = 0.9
# FeO_initial = lambda x: 11.3981 - 0.07896 * x["MgO"]
FeO_initial = 11.2
P_bar = 1

In [4]:
equilibrated_inclusions = mp.Melt(index=inclusions.index, columns=inclusions.columns, dtype=float, units="wt. %", datatype="oxide")
ol_equilibrated = pd.Series(index=inclusions.index, dtype=float)
temperatures = pd.Series(index=inclusions.index, dtype=float)
Kds = pd.Series(index=inclusions.index, dtype=float)

totalItems= equilibrated_inclusions.shape[0]
for i, ((idx, mi), (_, ol)) in enumerate(zip(inclusions.iterrows(), ol_host.iterrows())):

    print(f"{i + 1:0>3}/{totalItems:0>3} inclusions equilibrated", end= "\r")
    x, oc, t, kd = pec.Fe_equilibrate(mi, ol, P_bar)
    # print(f"{abs(t['old'] - t['new']):.2f}", end="\n")
    equilibrated_inclusions.loc[idx] = x.iloc[-1]
    ol_equilibrated.loc[idx] = oc[-1] 
    Kds.loc[idx] = kd["Real"]

In [5]:
corrected_inclusions = mp.Melt(index=inclusions.index, columns=inclusions.columns, dtype=float, units="wt. %", datatype="oxide")
ol_crystallised = pd.Series(index=inclusions.index, dtype=float)

totalItems = equilibrated_inclusions.shape[0]

for i, ((idx, mi), (_, ol)) in enumerate(zip(equilibrated_inclusions.iterrows(), ol_host.iterrows())):

    print(f"{i + 1:0>3}/{totalItems:0>3} inclusions corrected", end= "\r")

    x2, oc2, kd2, t = pec.crystallisation_correction(mi, ol, FeO_initial, P_bar)
    corrected_inclusions.loc[idx] = x2.iloc[-1]
    ol_crystallised.loc[idx] = oc2[-1]

In [12]:
((ol_crystallised + ol_equilibrated) - OC) * 100

name
PI032-01-01   -0.158112
PI032-01-04    0.345496
PI032-02-01    0.375070
PI032-02-02    0.356040
PI032-02-03   -0.360626
PI032-03-01    0.375065
PI032-04-01    0.560580
PI032-04-02    0.360329
PI032-04-03    0.351454
PI032-05-01    0.351209
PI041-02-01    0.000000
PI041-02-02   -0.352397
PI041-03-01    0.505680
PI041-03-03    0.160673
PI041-04-02    0.004215
PI041-04-03   -0.355376
PI041-04-04    0.347404
PI041-04-05   -0.285635
PI041-04-06   -0.349409
PI041-04-07    0.006090
PI041-05-02    0.006654
PI041-05-03   -0.081453
PI041-05-04    0.358751
PI041-05-06    0.352705
PI041-07-01    0.359898
PI041-07-02    0.356463
PI041-07-06   -0.009028
PI052-01-02    0.280524
PI052-02-01    0.342952
dtype: float64

In [10]:
OC * 100

name
PI032-01-01    12.0528
PI032-01-04     6.1552
PI032-02-01    11.9632
PI032-02-02    11.0592
PI032-02-03    12.5824
PI032-03-01    12.0848
PI032-04-01    11.1712
PI032-04-02    12.1648
PI032-04-03    10.5024
PI032-05-01     3.2864
PI041-02-01     1.6000
PI041-02-02     3.9696
PI041-03-01    13.9328
PI041-03-03    13.2176
PI041-04-02     3.5648
PI041-04-03     4.4288
PI041-04-04     2.7072
PI041-04-05     4.1056
PI041-04-06     4.4224
PI041-04-07     5.3648
PI041-05-02     0.5584
PI041-05-03     1.0480
PI041-05-04     1.4528
PI041-05-06     1.1072
PI041-07-01     9.4832
PI041-07-02     9.7008
PI041-07-06    10.1248
PI052-01-02    -1.4800
PI052-02-01    10.5136
dtype: float64

In [7]:
mi_pec = pec.PEC_olivine(inclusions, ol_host, P_bar, FeO_initial)

In [14]:
import MagmaPandas.geochemistry.fO2 as fO2
import MagmaPandas.geochemistry.Fe_redox as fe

T_K = x.temperature(P_bar)
moles = x.moles
fO2 = fO2.fO2_QFM(1, T_K, P_bar)
Fe3Fe2 = fe.Fe_redox.kressCarmichael(moles, T_K, fO2, P_bar)

Fe2_FeTotal = 1 / (1 + Fe3Fe2)
melt_MgFe = moles["MgO"] / (moles["FeO"] * Fe2_FeTotal)

Kd_eq, Kd_obs = mi_pec.calculate_Kds()

olivine_MgFe = melt_MgFe / Kds
Fo = olivine_MgFe / (olivine_MgFe + 1)

In [15]:
abs(Fo * 100 - ol_host.forsterite * 100)

sample
PI032-01-01    0.657957
PI032-01-04    0.436673
PI032-02-01    0.782766
PI032-02-02    0.634068
PI032-02-03    0.704613
PI032-03-01    0.710995
PI032-04-01    0.219759
PI032-04-02    0.208411
PI032-04-03    0.344535
PI032-05-01    0.523263
PI041-02-01    0.252472
PI041-02-02    0.472328
PI041-03-01    0.530517
PI041-03-03    0.558839
PI041-04-02    0.479181
PI041-04-03    0.451842
PI041-04-04    0.445859
PI041-04-05    0.559108
PI041-04-06    0.528314
PI041-04-07    0.639893
PI041-05-02    0.474424
PI041-05-03    0.505950
PI041-05-04    0.484916
PI041-05-06    0.416821
PI041-07-01    0.365196
PI041-07-02    0.413952
PI041-07-06    0.400915
PI052-01-02    0.461865
PI052-02-01    0.358838
dtype: float64

In [8]:
x, OC, T = mi_pec.correct()

029/029 inclusions equilibrated
029/029 inclusions corrected


In [ ]:
oc

In [ ]:
t = mi_pec.correct_inclusion(5)

In [ ]:
markersize = 12

fig, ax = plt.subplots(figsize=(8,7))

for (idx, i), (_, j) in zip(mi_pec.inclusions.iterrows(), mi_pec.inclusions_uncorrected.iterrows()):
    data = pd.concat([pd.Series(i), pd.Series(j)], axis=1).T
    data.index = ["corrected", "original"]
    plt.plot(data["MgO"], data["FeO"], "--", color="k")

plt.plot(mi_pec.inclusions["MgO"], mi_pec.inclusions["FeO"], "o", markersize=markersize)
plt.plot(mi_pec.inclusions_uncorrected["MgO"], mi_pec.inclusions_uncorrected["FeO"], "D", markersize=markersize)

ax.set_xlabel("wt. % MgO")
ax.set_ylabel("wt. % FeO")

plt.show()